### 方法
```
while 調べるべき特徴量がある限り :
    for 調べる特徴量 in 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいXsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量からXsを除外（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ

### データ
- mushroomの50サンプル分

### yのサンプリング
- 条件に合うまでサンプルを作り続ける

In [1]:
#!pip3 install dwave-neal

In [2]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import random

In [3]:
X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0)
y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

In [4]:
num_samples

50

In [5]:
y_sum

26

In [6]:
def make_y(minP_cols):
    while True:
        y = [[0, 1][random.randint(0, 1)]for jj in range(num_samples)]
        if sum(y) - y_sum != 0:
            continue
        if len(minP_cols) == 0:
            return pd.Series(y)
        else:
            for i in minP_cols:
                if np.dot(X_mushroom[i], y) - np.dot(X_mushroom[i], y_mushroom) != 0:
                    continue
            return pd.Series(y)

In [7]:
minP_cols = []
servey_num_col_list = columns
tt = 0

while len(servey_num_col_list) > 0:    
    
    p = {}
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp
        
        ti_dic = {} #{内積の値:該当数}という辞書。
        for _ in range(100): 
            y = make_y(minP_cols)
            
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1
                
        if not ori_ti in ti_dic.keys():
            ti_dic[ori_ti] = 1
            
        ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
        
        franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
        denomi = sum([j[1] for j in ti_list])
        p[i] = franc/denomi
    
    print('I am here')
    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]
    
    tt += 1
    print(tt)

I am here
1
I am here
2
I am here
3
I am here
4
I am here
5
I am here
6
I am here
7
I am here
8
I am here
9
I am here
10
I am here
11
I am here
12
I am here
13
I am here
14
I am here
15
I am here
16
I am here
17
I am here
18
I am here
19
I am here
20
I am here
21
I am here
22
I am here
23
I am here
24
I am here
25
I am here
26
I am here
27
I am here
28
I am here
29
I am here
30
I am here
31
I am here
32
I am here
33
I am here
34
I am here
35
I am here
36
I am here
37
I am here
38
I am here
39
I am here
40
I am here
41
I am here
42
I am here
43
I am here
44
I am here
45
I am here
46
I am here
47
I am here
48
I am here
49


In [8]:
minP_cols

['1',
 '6',
 '21',
 '27',
 '38',
 '56',
 '60',
 '94',
 '100',
 '108',
 '65',
 '111',
 '78',
 '74',
 '37',
 '45',
 '50',
 '88',
 '107',
 '99',
 '76',
 '67',
 '83',
 '112',
 '44',
 '18',
 '31',
 '113',
 '52',
 '97',
 '49',
 '14',
 '25',
 '58',
 '51',
 '13',
 '118',
 '22',
 '64',
 '115',
 '106',
 '101',
 '43',
 '41',
 '54',
 '2',
 '109',
 '5',
 '117',
 '69',
 '114',
 '19',
 '53',
 '9',
 '47',
 '8',
 '12',
 '35',
 '68',
 '87',
 '0',
 '3',
 '4',
 '7',
 '10',
 '11',
 '15',
 '16',
 '17',
 '20',
 '23',
 '24',
 '26',
 '28',
 '29',
 '30',
 '32',
 '33',
 '34',
 '36',
 '39',
 '40',
 '42',
 '46',
 '48',
 '55',
 '57',
 '59',
 '61',
 '62',
 '63',
 '66',
 '70',
 '71',
 '72',
 '73',
 '75',
 '77',
 '79',
 '80',
 '81',
 '82',
 '84',
 '85',
 '86',
 '89',
 '90',
 '91',
 '92',
 '93',
 '95',
 '96',
 '98',
 '102',
 '103',
 '104',
 '105',
 '110',
 '116']